In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
import random
import dataclasses
import jax
import optax

import jax.numpy as jnp
import numpy as np

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.utils.load_sae import get_sae
from functools import partial

In [3]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

Cloning into 'itv'...
fatal: unable to access 'https://github.com/roeehendel/icl_task_vectors data/itv/': URL using bad/illegal format or missing URL


In [6]:
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [7]:
def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs

In [8]:
prompt = "<user>Follow the pattern\n{}"

In [9]:

target_layer = 17

In [10]:
task_names = [
    "en_es"
]

task = "en_es"

n_seeds = 10

# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 32, 64, 256

In [11]:
def get_logprob_diff(logits: jnp.ndarray, target_tokens, print_results=False):
    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            tokenizer.decode(answer_logprobs.argmax(axis=-1))
        )

        print(
            tokenizer.decode(target_tokens)
        )

    return target_logprobs


In [12]:
from micrlhf.llama import LlamaBlock
from functools import partial

def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids

In [13]:
import dataclasses
def get_loss(weights, dictionary, inputs, target_tokens, taker, initial_resids):
    weights = jax.nn.relu(weights)
    

    recon = jnp.einsum("fv,f->v", dictionary, weights)
    recon = recon.astype('bfloat16')

    modified = pz.nx.nmap(lambda a, b: a.at[-1].add(b))(
        initial_resids.untag("seq", "embedding"), pz.nx.wrap(recon, "embedding").untag("embedding")
        ).tag("seq", "embedding")

    inputs = dataclasses.replace(inputs, tokens=modified)

    logits = taker(inputs).unwrap("batch", "seq", "vocabulary")

    logprob_diff = get_logprob_diff(logits, target_tokens)
    loss = -logprob_diff.mean()

    return loss + 2e-2 * jnp.linalg.norm(weights, ord=1), ((weights != 0).sum(), loss)
    # return loss, ((weights != 0).sum(), loss)

In [14]:
def train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, initial_resids, optimizer, lwg, pos_only=True):
    (loss, (l0, loss_)), grad = lwg(weights, dictionary, inputs, target_tokens, taker, initial_resids)

    updates, opt_state = optimizer.update(grad, opt_state, weights)
    weights = optax.apply_updates(weights, updates)
    # weights_abs = jnp.abs(weights)
    # weights = jnp.sign(weights) * jax.nn.relu(weights_abs - shrinkage)

    return loss, weights, opt_state, dict(l0=l0, loss=loss_)

In [15]:
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)

In [16]:
from task_vector_utils import FeatureSearch

In [17]:
import datasets
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", "en-es", split="validation")

In [18]:
sample = dataset.shuffle()[:200]
sample = sample["translation"]

In [19]:
pairs = [(x["en"], x["es"]) for x in sample]
# pairs

In [38]:
picked_features = jnp.array(list(set([
    604, 34292,  6661, 24549,  6198, 14166, 44525, 41973, 35027,
       15949, 18911, 13590, 40721, 45024,  8023, 14153, 35597, 36343,
       24992, 23008, 34292, 24549, 6198, 14166, 44525, 41973, 18911, 13590
])))
fs = FeatureSearch("en_es", pairs, 18, l1_coeff=2e-2, init_w=0.5, lr=1e-2, n_shot=1, n_first=3, sae_v=4, early_stopping_steps=2000, max_seq_len=500, picked_features=picked_features)
weights, metrics = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
pairs = tasks["en_es"]
pairs = list(pairs.items())

In [20]:
from micrlhf.utils.vector_storage import load_vector
tv = load_vector("task_vectors/en_es:18")

In [22]:
from micrlhf.utils.load_sae import get_sae, sae_encode_gated

sae = get_sae(18, 4)

--2024-05-28 13:17:59--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-4-8.86E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/fa68513c10a8cdd065e4a0e66c05816325e4d72fb272857ca70564fca7fa808f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717161479&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzE2MTQ3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZmE2ODUxM2

In [23]:
_, post_relu, _ = sae_encode_gated(sae, tv)

In [26]:
jax.lax.top_k(post_relu, 40)

[# jax.Array float32(40,) ≈5.6 ±2.3 [≥2.8, ≤1.4e+01] nonzero:40
   Array([14.33634  ,  9.284671 ,  9.173065 ,  9.019746 ,  8.390729 ,
           8.324912 ,  7.921408 ,  7.3480153,  7.305516 ,  7.0247445,
           6.195833 ,  6.159084 ,  6.1102242,  6.035726 ,  5.982573 ,
           5.932697 ,  5.9004   ,  5.7842216,  5.478712 ,  5.34142  ,
           4.71809  ,  4.631511 ,  4.5113354,  4.456746 ,  4.3027215,
           4.2715216,  4.2645254,  4.1496553,  3.8654673,  3.8076172,
           3.7399244,  3.703537 ,  3.6620202,  3.5615187,  3.1690292,
           3.1292303,  3.050812 ,  2.9940195,  2.8714151,  2.7897758],      dtype=float32)
 ,
 # jax.Array int32(40,) [≥720, ≤48_437] nonzero:40
   Array([ 1065, 24363, 46849, 33278, 44525, 35202, 48437, 14577, 24549,
          33039, 47882, 39179,  8787, 46421, 33953, 22038, 33045, 48299,
          28517, 34138, 41973,   720, 16422,  9224, 43484, 28188, 11831,
          11950,  7677,  8143, 43555, 46370, 26046, 45228, 14166, 26164,
          39233,  1699,  5405,  9218], dtype=int32)
 ]

In [30]:
fs = FeatureSearch("en_es", pairs, 18, l1_coeff=2e-2, init_w=post_relu, lr=1e-2, n_shot=1, n_first=2, sae_v=4, early_stopping_steps=300, max_seq_len=500, seed=111)
weights, metrics = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [32]:
jax.lax.top_k(weights, 25)

[# jax.Array float32(25,) ≈4.2 ±3.4 [≥0.0, ≤1.3e+01] zero:3 nonzero:22
   Array([12.591403  , 11.225099  , 10.420396  ,  6.5521755 ,  6.3882265 ,
           6.2229795 ,  6.222977  ,  6.054167  ,  6.014406  ,  4.9479194 ,
           3.8334608 ,  3.8307266 ,  3.6801434 ,  3.4142656 ,  3.074797  ,
           2.402644  ,  2.034124  ,  1.6284066 ,  1.3841197 ,  1.3459913 ,
           0.54155225,  0.5363517 ,  0.        ,  0.        ,  0.        ],      dtype=float32)
 ,
 # jax.Array int32(25,) [≥0, ≤48_437] zero:1 nonzero:24
   Array([ 1065, 44525, 24549,   720, 41973, 35202, 33039, 15865,  9218,
          14166, 13590, 36287, 46849,  9014, 24363, 19597, 48437, 14577,
          47882, 33045, 33278, 22038,     0,     1,     2], dtype=int32)
 ]

In [31]:
results2 = (weights, metrics)

In [ ]:
results1 = (weights, metrics)

In [25]:
jax.lax.top_k(weights, 20)

[# jax.Array float32(20,) ≈5.3 ±4.1 [≥-0.01, ≤1.5e+01] nonzero:20
   Array([ 1.4764973e+01,  1.1060773e+01,  1.0521282e+01,  8.4687090e+00,
           7.9338636e+00,  7.1173511e+00,  7.0394602e+00,  6.9095325e+00,
           6.7327070e+00,  6.3283100e+00,  5.1446500e+00,  5.0161490e+00,
           3.7937768e+00,  2.7460663e+00,  1.8403769e+00,  3.5122186e-01,
          -5.3760563e-03, -9.4772000e-03, -9.7897071e-03, -1.0356248e-02],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥604, ≤45_024] nonzero:20
   Array([  604, 34292,  6661, 24549,  6198, 14166, 44525, 41973, 35027,
          15949, 18911, 13590, 40721, 45024,  8023, 14153, 35597, 36343,
          24992, 23008], dtype=int32)
 ]

In [33]:
jax.lax.top_k(jnp.abs(results2[0]), 10)

[# jax.Array float32(10,) ≈2.2e+01 ±1.5e+01 [≥0.84, ≤4.5e+01] nonzero:10
   Array([45.142223  , 42.201675  , 34.756268  , 31.79385   , 25.685295  ,
          12.871383  , 12.606635  ,  7.3319654 ,  3.9815216 ,  0.84317565],      dtype=float32)
 ,
 # jax.Array int32(10,) [≥4_218, ≤97_588] nonzero:10
   Array([58435, 97588, 42667,  7229, 18631, 50610, 41969, 89010,  4218,
           4316], dtype=int32)
 ]

In [37]:
pairs

[('Deprivation of liberty of migrants must comply not only with national law, but also with international legislation.',
  'La privación de la libertad de los migrantes debe cumplir no sólo con las disposiciones del derecho nacional, sino también con la legislación internacional.'),
 ('- You nuts?', '- ¿Eres idiota o qué?'),
 ('Actually, love is a strange thing.',
  'En estos momentos, el amor es algo extraño.'),
 ("Tony's frustrated I didn't cry.", 'Tony está frustrado porque no lloré.'),
 ('- Yeah? Listen, I got a subpoena here for you to appear in federal court tomorrow to reveal the source of the Delorean tape.',
  'El juez te ordenó revelar la fuente de la cinta del FBI.'),
 ('Map of Noorddorp .', 'Mapa de Noorddorp .'),
 ('No need to, Ida.', 'No es necesario, Ida.'),
 ('I still think you should bow out of this anatomy class you signed up to teach.',
  'Sigo pensando que deberías retirarte de esa clase de anatomía que accediste a dar.'),
 ('Attention Cinderellas, one and all.', 'Atención Cenicientas, una y todas.'),
 ('It was therefore a matter of regret that only eight countries, including Japan, had thus far ratified the Additional Protocol to the Safeguards Agreements.',
  'Por lo tanto, cabe lamentar que sólo ocho países, incluido el Japón, hayan ratificado hasta ahora el Protocolo Adicional a los acuerdos de salvaguardias.'),
 ('_BAR_ Article 2 3 3 — Total _BAR_ 30000 _BAR_ _BAR_ _BAR_',
  '_BAR_ Total del artículo 2 3 3 _BAR_ 30000 _BAR_ _BAR_ _BAR_'),
 ('- Happy Thanksgiving, Brenda. - Happy Thanksgiving',
  '- Feliz día de Acción de Gracias.'),
 ('- Mother, can you--', '- Madre, podrías...'),
 ('So, why do Europe’s leaders find this intermediate solutionso unimaginable?',
  'Así que, ¿por qué los líderes europeos encuentran taninimaginable esta solución intermedia?'),
 ("Now there's the bastard I know and love.",
  'Ese es el desgraciado que conozco y que amo.'),
 ('You can win all the contests.', 'Puedes ganar todos los concursos.'),
 ('Ref: 806', 'Ref: 1945'),
 ('- I thought this was the bathroom.', 'Pensé que éste era el baño.'),
 ("No, 'cause I think the F1, it only covers the bloodwork.",
  'No, porque creo que la F1, solo cubre el trabajo sanguíneo.'),
 ('The articles of before 1996 are now being digitized, indexed and placed on the internet.',
  'Las propuestas se deben enviar antes del 15 de Noviembre de 2012, y deben estar referidas al tema descrito anteriormente.'),
 ('Draft resolution A/C.3/60/L.34 was adopted by 121 to 52.',
  'Por 121 votos contra 52 queda aprobado el proyecto de resolución A/C.3/60/L.34.'),
 ('Under the Arms and Explosives Act 2000, any person who wishes to import firearms has to apply for permit from the Royal Brunei Police Force.',
  'Con arreglo a lo dispuesto en la Ley de armas y explosivos de 2000, toda persona que desee importar armas de fuego deberá solicitar un permiso de la Real Fuerza de Policía de Brunei.'),
 ('San Antonio.', 'San Antonio.'),
 ('Applicants who declare that they have been involved in such activities are liable to have their visas refused under section 501 of the Migration Act 1958 or Article 1F of the Convention in the protection visa application context.',
  'En virtud del artículo 501 de la Ley de Inmigración de 1958 o del artículo 1F de la Convención, se puede rechazar la solicitud de visado de los solicitantes que declaren que han participado en actividades de esa índole, en el contexto de la solicitud de visado de protección.'),
 ('I would find it most regrettable if it was to be put into practice at the end of the day.',
  'Me parecería lamentable que finalmente fuese puesta en práctica.'),
 ('tioxidazole', 'tioxidazol'),
 ("If there's nothing else I can do, I think I'll turn in.",
  '- Si no hay nada más que hacer, voy a dormir - Cuida de mi caballo.'),
 ('You t-- You dated Addison, too?', '¿Tú... también saliste con Addison?'),
 ("- I'll buy some stuff and bring it over.", '- Comprare algo y lo llevare.'),
 ('When a criminal has caused injury to som

In [34]:

i = jax.lax.top_k(jnp.abs(results1[0]), 10)[1]
fs.check_features(i, scale=20)

  0%|          | 0/10 [00:00<?, ?it/s]

(<jax.Array(28380, dtype=int32)>,
 <jax.Array(46.75, dtype=bfloat16)>,
 <jax.Array(47.75, dtype=bfloat16)>,
 <jax.Array([46.75, 47.75, 47.75, 47.75, 47.5, 48.25, 47.5, 48, 47.5, 47.75],      dtype=bfloat16)>)

In [35]:

i = jax.lax.top_k(jnp.abs(results2[0]), 10)[1]
fs.check_features(i, scale=20)

  0%|          | 0/10 [00:00<?, ?it/s]

(<jax.Array(4218, dtype=int32)>,
 <jax.Array(47.25, dtype=bfloat16)>,
 <jax.Array(47.5, dtype=bfloat16)>,
 <jax.Array([47.75, 47.5, 47.75, 47.5, 47.75, 47.5, 47.5, 47.75, 47.25, 48],      dtype=bfloat16)>)

In [39]:
fs = FeatureSearch("en_es", pairs, 18, l1_coeff=1e-2, init_w=0.5, lr=5e-2, n_shot=1, n_first=10, sae_v=4, early_stopping_steps=150)

In [40]:
w = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [41]:
_, i = jax.lax.top_k(w[0], 20)

In [42]:
fs.check_features(i, 20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(6198, dtype=int32)>,
 <jax.Array(61.75, dtype=bfloat16)>,
 <jax.Array(66, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈6.6e+01 ±1.1 [≥6.2e+01, ≤6.6e+01] nonzero:20
   Array([66.5, 66, 66.5, 66.5, 66, 66, 66, 66, 66, 61.75, 66.5, 64.5, 66.5,
          66.5, 66.5, 66.5, 66.5, 64.5, 66.5, 66], dtype=bfloat16)
 )

In [50]:
fs.check_feature(34107, 20)

<jax.Array(86, dtype=bfloat16)>

In [49]:
steering_vector = fs.sae["W_dec"][37312] * 40
steering_vector =  steering_vector.astype('bfloat16')

act_add = make_act_adder(
    llama, steering_vector, fs.eval_tokens, fs.target_layer, length=1
)


logits = act_add(fs.eval_inputs).unwrap("batch", "seq", "vocabulary")

In [50]:
positions = jnp.argwhere(fs.eval_tokens == 1599)[:, -1]


In [36]:
_logits = jax.vmap(
    lambda a, b: a[b]
)(logits, positions)
tokenizer.decode(_logits.argmax(axis=-1))

'down walk scene small hell scene ground film game sit method fe song enjoy definition in ent ag nation mad dog art tax oficial prote pages of life space golf union political'

In [37]:
fs.runner.eval_pairs

[[['down', 'abajo']],
 [['walk', 'caminar']],
 [['scene', 'escena']],
 [['large', 'grande']],
 [['hell', 'infierno']],
 [['scene', 'escena']],
 [['ground', 'tierra']],
 [['film', 'película']],
 [['stage', 'escenario']],
 [['sit', 'sentarse']],
 [['method', 'método']],
 [['pretty', 'bonito']],
 [['song', 'canción']],
 [['enjoy', 'disfrutar']],
 [['term', 'término']],
 [['into', 'en']],
 [['understand', 'comprender']],
 [['agency', 'agencia']],
 [['nation', 'nación']],
 [['hard', 'duro']],
 [['dog', 'perro']],
 [['art', 'arte']],
 [['tax', 'impuesto']],
 [['official', 'oficial']],
 [['protect', 'proteger']],
 [['page', 'página']],
 [['off', 'apagado']],
 [['life', 'vida']],
 [['space', 'espacio']],
 [['club', 'club']],
 [['union', 'unión']],
 [['political', 'político']]]

In [66]:
fs.eval_tokens

array([[32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3204,  1599,   633,  7069],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20919,  1599,  3949, 18220],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 24645,  1599,  3966,  2386],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 16961,  1599,  6815],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 14181,  1599,  3041, 12643],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 24645,  1599,  3966,  2386],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  2057,  1599, 27907],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  9663,  1599, 19053],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 19190,  1599,  3966, 24893],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 29879,   277,  1599,  2665,  7989],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  5696,  1599, 11510,  8144],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  1457,  4349,  1599, 10814,  2049],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 21453,  1599, 24174],
       [    1, 32010, 10306,   278,  4766, 29901,    13,   264,  2212,
        29891,  1599,   766,  1341,   329,   279],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  8489,  1599, 27757, 29877],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,  8941,  1599,   427],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  5062,  1689,  1599,  7199,  1581],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,   351,  3819,  1599,   946,  5760],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 29876,   362,  1599,  1055,  1290],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  6800,  1599,   868,   307],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 26169,  1599,   639,   307],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13,   442,  1599, 20160],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20725,  1599,  2411, 29771],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 29877,  7880,  1599,  9535],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13, 14676,   312,  1599,  3279,   914],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3488,  1599,   282, 18814],
       [32000, 32000, 32000,     1, 32010, 10306,   278,  4766, 29901,
           13,  2696,  1599,  3095,   351,   912],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 19264,  1599, 10828],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13,  3493,  1599,  9015,  3934],
       [32000, 32000, 32000, 32000, 32000,     1, 32010, 10306,   278,
         4766, 29901,    13, 29066,  1599,  4402],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 13094,  1599,   443,  3175],
       [32000, 32000, 32000, 32000,     1, 32010, 10306,   278,  4766,
        29901,    13, 20087,   936,  1599, 24934]])

In [61]:
t

'down walked screen larger hello screen ground film stage sat method pu song enjoy value into understanding ag nation harder log art tax off protect pages off life space club union political'

In [44]:
def check_feature(feature: int, task:str, target_layer, sae, print_results=False):
    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=10, prepend_space=task.startswith("algo"))

    print(
        dataset.prompts
    )


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    pos = 1
    if task.startswith("algo"):
        pos = 2

    target_tokens = [x[pos] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    # weights, recon = grad_pursuit(tv * 2, sae["W_dec"], k, pos_only=True)

    recon = sae["W_dec"][feature] * 20
    recon = recon.astype('bfloat16')

    act_add = add_vector(
        llama, recon, target_layer, scale=1.0, position="last"
    )

    logits = act_add(inputs).unwrap("batch", "seq", "vocabulary")

    # print(logits[:, -1].mean(axis=-1))

    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            repr(tokenizer.decode(answer_logprobs.argmax(axis=-1)))
        )

        print(
            repr(tokenizer.decode(target_tokens))
        )

    return (target_logprobs - answer_logprobs.max(axis=-1))

In [27]:
sae = get_sae(18, 4)

In [30]:
sae

{'W_dec': # jax.Array float32(49152, 3072) ≈1.5e-05 ±0.018 [≥-0.69, ≤0.72] nonzero:150_994_944
   Array([[-1.0390111e-02, -1.4213075e-02, -2.7305655e-02, ...,
           -3.8054176e-02,  3.2431733e-02, -2.4696782e-02],
          [ 3.6464450e-03,  1.1525472e-02,  2.3346700e-02, ...,
           -1.3969769e-02, -2.1684350e-02,  1.9762961e-02],
          [ 2.3951381e-02,  2.0307090e-02, -1.0205399e-03, ...,
           -7.8665420e-05, -3.2605302e-02, -2.0566324e-02],
          ...,
          [-2.1444883e-02, -5.0013666e-03, -5.5012279e-03, ...,
            1.4278403e-02,  1.3525741e-02, -3.5212745e-04],
          [ 2.8317727e-02,  2.8130701e-02,  7.6365019e-03, ...,
           -9.8304339e-03,  7.1255758e-04, -2.6758818e-02],
          [ 2.5761945e-04, -9.7675147e-03, -1.6709493e-02, ...,
           -8.7964898e-03,  4.5935563e-03,  1.5758382e-02]], dtype=float32)
 ,
 'W_enc': # jax.Array float32(3072, 49152) ≈2.7e-05 ±0.015 [≥-0.33, ≤0.75] zero:1 nonzero:150_994_943
   Array([[ 0.00642347,  0.01031775,  0.00772383, ..., -0.01876058,
            0.06085767, -0.01564161],
          [-0.00819698, -0.01002671,  0.00421169, ..., -0.02776144,
            0.03665377, -0.03213882],
          [ 0.00030221,  0.01530008, -0.00118439, ...,  0.00538867,
            0.02733457, -0.00622977],
          ...,
          [ 0.00109215, -0.00682703, -0.00306622, ...,  0.03215252,
           -0.03750627, -0.0165697 ],
          [ 0.01241016, -0.00268804, -0.0145798 , ...,  0.01233974,
           -0.0092611 , -0.01979413],
          [-0.00127209,  0.00695627, -0.00180823, ..., -0.01949977,
           -0.02877844,  0.01321251]], dtype=float32)
 ,
 'b_dec': # jax.Array float32(3072,) ≈0.00065 ±0.063 [≥-0.44, ≤0.61] nonzero:3_072
   Array([ 0.00894668,  0.00368893, -0.00755703, ...,  0.03014987,
           0.07653498,  0.06849926], dtype=float32)
 ,
 'b_enc': # jax.Array float32(49152,) ≈-0.13 ±0.16 [≥-0.76, ≤0.11] nonzero:49_152
   Array([-0.02901577, -0.07799948, -0.09279591, ..., -0.5452185 ,
          -0.42847958, -0.3660079 ], dtype=float32)
 ,
 'b_gate': # jax.Array float32(49152,) ≈0.19 ±0.27 [≥0.01, ≤1.2] nonzero:49_152
   Array([0.02074471, 0.08082295, 0.20455228, ..., 0.9924683 , 0.7404377 ,
          0.5968205 ], dtype=float32)
 ,
 's_gate': # jax.Array float32(49152,) ≈0.084 ±0.13 [≥-0.82, ≤0.57] nonzero:49_152
   Array([ 0.01797964,  0.09803171,  0.28549254, ..., -0.16841878,
           0.18351474,  0.27063075], dtype=float32)
 ,
 'scaling_factor': # jax.Array float32(49152,) ≈1.0 ±0.058 [≥0.51, ≤1.9] nonzero:49_152
   Array([1.0194339, 1.1125576, 0.994432 , ..., 1.2642885, 1.0430051,
          1.0159718], dtype=float32)
 }

In [45]:
sae = get_sae(18, 4)

check_feature(6198, "en_es", 18, sae, print_results=True)

['worry ->', 'money ->', 'hear ->', 'general ->', 'morning ->', 'government ->', 'course ->', 'letter ->', 'oil ->', 'future ->', 'kill ->', 'cut ->', 'risk ->', 'time ->', 'student ->', 'wear ->', 'organization ->', 'simply ->', 'population ->', 'doctor ->', 'attention ->', 'money ->', 'carry ->', 'require ->', 'court ->', 'fund ->', 'summer ->', 'something ->', 'environment ->', 'industry ->', 'reduce ->', 'player ->', 'money ->', 'subject ->', 'dead ->', 'comment ->', 'position ->', 'military ->', 'art ->', 'great ->', 'town ->', 'make ->', 'bill ->', 'fine ->', 'love ->', 'clear ->', 'drug ->', 'foreign ->', 'administration ->', 'discuss ->', 'watch ->', 'recognize ->', 'gun ->', 'gun ->', 'guy ->', 'rule ->', 'on ->', 'ok ->', 'club ->', 'much ->', 'situation ->', 'respond ->', 'cut ->', 'provide ->']


'wor mon here general tarde government curs number pet fut k cort ries  estud vest organiz s población enfer at mon c require co fund ver something natur indust reduce jug mon verb death <  militar art grande town me bill fin am cl dro for administ disc mir reconoc g g ch  en ok _ mucho action res cort segu'
'pre din esc general ma gobierno cur cart ace fut mat cort ries h estud lle organiz sim población méd at din lle ex corte fond ver algo amb indust redu jug din sujet m com pos militar arte est pueblo fabric fact mult amor clar dro extr admin disc re reconoc p p ch reg sobre ok club mucho situ res cort prove'


# jax.Array bfloat16(64,) ≈-2.5 ±3.1 [≥-1.2e+01, ≤0.0] zero:20 nonzero:44
  Array([-3, -2.75, -9.5625, 0, -1.625, -0.5, -1.75781, -12.0625, -2.1875,
         0, -2.75, 0, 0, -1.9375, 0, -0.5, 0, -1.11719, 0, -1.375, 0, -2.75,
         -1.25, -8.125, -1, -1.11719, 0, -1.625, -0.0625, 0, -0.121094, 0,
         -2.75, -4.6875, -4.5, -3.01562, -3.25, 0, -3.39062, -7.0625,
         -5.375, -5.1875, -10.3125, -11.875, -0.996094, -0.617188, 0,
         -3.625, -1.375, 0, -7, 0, -3.8125, -3.8125, 0, -6, -5.25, 0,
         -0.9375, 0, -6.5, 0, 0, -4.6875], dtype=bfloat16)

In [ ]:
for feature in i:
    print(
        f"Feature {feature}: {check_feature(feature)}"
    )

# get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), target_tokens, print_results=True)

Feature 10018: -7.5
Feature 20311: -2.375
Feature 40023: -4.5
Feature 5145: -7.21875
Feature 45145: -7.09375
Feature 2719: -7.09375
Feature 15415: -7.28125
Feature 24802: -7.15625
Feature 20375: -7.125
Feature 19505: -7.09375
Feature 38618: -7.25
Feature 36786: -7.125
Feature 47611: -7.03125
Feature 31726: -7.28125


KeyboardInterrupt: 

In [ ]:
check_feature(764, True)

message reduce behavior place se hello cel media club live line throw sport es door shape daughter run pu
 bottom mind mat polit year moment cut hour has sit comment drop un government d wait phone anticip south arm sport material place picture shape papel no en < choice viel fine arrived ii f third pu hang u er must just hello fig
mess rid comport lu sic infer cell media club abit punto get sport così porta col fig corr bel reg super m question polit dec momento tag ora aver sed comment g gi governo pap asp tele asp sud es sport mater lu f col cart no met rela sc pi bene arriv io cin ter bel append fu dire do app infer figura


<jax.Array(-5.9375, dtype=bfloat16)>

In [17]:
fs = FeatureSearch("algo_last", 18)

--2024-05-23 13:29:53--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.51, 108.156.211.95, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716730193&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjczMDE5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

In [18]:
ds = fs.eval_dataset

In [18]:
info = """location_continent 21 -1.09375
football_player_position 21 -3.76562
location_religion 21 -1.09375
location_language 20 -0.457031
person_profession 21 -1.30469
location_country 21 -1.44531
country_capital 18 -1.25781
person_language 18 -0.287109
singular_plural 21 -0.261719
present_simple_past_simple 20 -0.341797
antonyms 14 -0.75
plural_singular 20 -0.133789
present_simple_past_perfect 18 -1.25781
present_simple_gerund 20 -0.171875
en_it 18 -1.29688
it_en 14 -1.75781
en_fr 18 -1.39062
en_es 18 -1.21875
fr_en 17 -1.25781
es_en 17 -1.09375
en_ru 18 ~1.1
en_de 18 ~1
algo_max 24 -0.515625
algo_min 13 -0.945312
algo_last 17 -0.162109
algo_first 18 -0.828125|
algo_sum 14 -1.49219
algo_most_common 12 -1.15625
"""

In [21]:
info = """location_continent: w/o: 7.96875, w/: 1.91406, layer: 25
football_player_position: w/o: 13.125, w/: 4.375, layer: 22
location_religion: w/o: 7.875, w/: 1.55469, layer: 27
location_language: w/o: 7.9375, w/: 1.82812, layer: 21
person_profession: w/o: 11.3125, w/: 3.53125, layer: 24
location_country: w/o: 5.625, w/: 3.67188, layer: 21
country_capital: w/o: 6.40625, w/: 3.73438, layer: 18
person_language: w/o: 7.4375, w/: 1.28906, layer: 21
singular_plural: w/o: 1.96875, w/: 0.910156, layer: 21
present_simple_past_simple: w/o: 2.48438, w/: 0.773438, layer: 20
antonyms: w/o: 5.96875, w/: 2.29688, layer: 18
plural_singular: w/o: 2.53125, w/: 1.53906, layer: 18
present_simple_past_perfect: w/o: 4.65625, w/: 2.375, layer: 21
present_simple_gerund: w/o: 3.51562, w/: 1.17188, layer: 21
en_it: w/o: 15.375, w/: 3.0625, layer: 18
it_en: w/o: 8.0625, w/: 3.8125, layer: 18
en_ru: w/o: 16.125, w/: 3.76562, layer: 21
en_fr: w/o: 12.625, w/: 5.6875, layer: 18
en_es: w/o: 14.625, w/: 4.65625, layer: 18
fr_en: w/o: 7.3125, w/: 2.45312, layer: 18
es_en: w/o: 8.5, w/: 1.34375, layer: 20
en_de: w/o: 12.125, w/: 6.9375, layer: 18
algo_max: w/o: 3.0625, w/: 2.73438, layer: 24
algo_min: w/o: 3, w/: 2.5, layer: 13
algo_last: w/o: 3.39062, w/: 2.53125, layer: 17
algo_first: w/o: 3.20312, w/: 2.57812, layer: 18
algo_sum: w/o: 5.25, w/: 4.71875, layer: 14
algo_most_common: w/o: 2.92188, w/: 2.67188, layer: 12
"""

In [22]:
tasks_to_check = [x.split(": ") for x in info.split("\n") if x]
tasks_to_check = [(x[0], int(x[-1])) for x in tasks_to_check]
# tasks_to_check = [(x[0], int(x[1])) for x in tasks_to_check]
tasks_to_check = [
    ("en_es", 18),
#     # ("en_it", 18),
#     # ("en_fr", 18),
#     # ("en_de", 18),
#     # ("en_ru", 18)
#     ("algo_last", 18),
#     ("algo_first", 18),
#     ("algo_sum", 14),
] 
# tasks_to_check = [
#     x for x in tasks_to_check if x[0].startswith("algo")
# ]

tasks_to_check

[('location_continent', 25),
 ('football_player_position', 22),
 ('location_religion', 27),
 ('location_language', 21),
 ('person_profession', 24),
 ('location_country', 21),
 ('country_capital', 18),
 ('person_language', 21),
 ('singular_plural', 21),
 ('present_simple_past_simple', 20),
 ('antonyms', 18),
 ('plural_singular', 18),
 ('present_simple_past_perfect', 21),
 ('present_simple_gerund', 21),
 ('en_it', 18),
 ('it_en', 18),
 ('en_ru', 21),
 ('en_fr', 18),
 ('en_es', 18),
 ('fr_en', 18),
 ('es_en', 20),
 ('en_de', 18),
 ('algo_max', 24),
 ('algo_min', 13),
 ('algo_last', 17),
 ('algo_first', 18),
 ('algo_sum', 14),
 ('algo_most_common', 12)]

In [23]:

results = {}

In [26]:
from datasets import load_dataset


for task, target_layer in tqdm(tasks_to_check):
    if target_layer == 21:
        target_layer = 20
    if target_layer == 14:
        target_layer = 12
    if target_layer == 13:
        target_layer = 12
    if target_layer == 25:
        target_layer = 24
    if target_layer == 27:
        target_layer = 24
    if target_layer == 22:
        target_layer = 24
    # if target_layer == 17:
    #     target_layer = 18

    if "en_" in task:
        l,r = task.split("_")

        binding = f"{l}-{r}"
        if task == "en_de":
            binding = "de-en"


        dataset = load_dataset("Helsinki-NLP/opus-100", binding, split="validation")
        sample = dataset.shuffle()[:40]
        sample = sample["translation"]
        pairs = [(x[l], x[r]) for x in sample]
    else:
        pairs = tasks[task]
        pairs = [list(x) for x in pairs.items()]


    fs = FeatureSearch(task, pairs, target_layer, l1_coeff=5e-3, init_w=0.5, lr=5e-2, n_shot=1, n_first=10, sae_v=8, early_stopping_steps=200, max_seq_len=500)
    weights, metrics = fs.find_weights()
    i = jax.lax.top_k(jnp.abs(weights), 30)[1]
    best_feature, best_loss, mean_loss, losses = fs.check_features(i, scale=20)

    metrics = {k:float(v) for k, v in metrics.items()}

    results[task] = (best_feature, best_loss, mean_loss, losses, metrics, i)
    print(
        f"Task {task}, best feature {best_feature}, best loss {best_loss}, mean loss {mean_loss}, metrics {metrics}"
    )

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Task en_ru, best feature 13504, best loss 66, mean loss 67, metrics {'l0': 22.0, 'loss': 59.75}


  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Task es_en, best feature 28802, best loss 5.09375, mean loss 7.53125, metrics {'l0': 24.0, 'loss': 1.4140625}


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Task en_de, best feature 86736, best loss 65.5, mean loss 66.5, metrics {'l0': 96.0, 'loss': 51.0}


In [22]:
results

{'algo_last': (<jax.Array(1100, dtype=int32)>,
  <jax.Array(4.125, dtype=bfloat16)>,
  <jax.Array(5.59375, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈5.6 ±0.62 [≥4.1, ≤6.8] nonzero:30
    Array([4.90625, 5.09375, 4.8125, 4.6875, 4.4375, 4.125, 5.1875, 5.125,
           5.90625, 5.84375, 5.25, 6.03125, 5.34375, 6.28125, 6.75, 6.34375,
           5.53125, 5.8125, 5.5625, 6.5, 5.34375, 5.625, 5.78125, 6.03125,
           5.78125, 6.125, 5.1875, 5.90625, 5.625, 6.53125], dtype=bfloat16)
  ,
  {'l0': 7.0, 'loss': 1.6640625},
  # jax.Array int32(30,) [≥1_100, ≤46_999] nonzero:30
    Array([32994, 33663,  1195, 20545, 18196,  1100, 46999, 18894, 17808,
           22994, 40716, 27662,  4539, 46077, 18324, 38203, 35712, 28566,
           31959, 38810, 15949, 23636, 39474, 35126, 25640, 17645, 11918,
           34983, 46905, 29439], dtype=int32)
  ),
 'algo_first': (<jax.Array(1100, dtype=int32)>,
  <jax.Array(3.6875, dtype=bfloat16)>,
  <jax.Array(4.59375, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈4.6 ±0.68 [≥3.7, ≤6.5] nonzero:30
    Array([3.6875, 3.89062, 3.6875, 4.375, 4.21875, 4.59375, 4.53125, 4.46875,
           4.5625, 4.59375, 6.53125, 4.0625, 4.25, 3.89062, 3.96875, 4.53125,
           4.59375, 6.46875, 3.85938, 4.03125, 4.90625, 4.5, 4.4375, 5.21875,
           4.625, 4.65625, 5.46875, 5.09375, 5.21875, 4.9375], dtype=bfloat16)
  ,
  {'l0': 21.0, 'loss': 0.3671875},
  # jax.Array int32(30,) [≥1_100, ≤47_859] nonzero:30
    Array([ 1100, 22716, 20545, 46681, 40820,  3250,  5723, 46999, 21126,
           47859, 35201, 26589, 47002, 23052, 13656, 36343, 30385, 37997,
           15688, 35596, 10097, 33038, 41973,  7532, 40716, 14238,  1286,
           20265, 31959, 37752], dtype=int32)
  ),
 'algo_sum': (<jax.Array(26714, dtype=int32)>,
  <jax.Array(10, dtype=bfloat16)>,
  <jax.Array(11.875, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈1.2e+01 ±0.98 [≥1e+01, ≤1.4e+01] nonzero:30
    Array([12.125, 12.0625, 13.0625, 10.5, 11.8125, 10.4375, 11.1875, 11.125,
           10.5625, 11.25, 13.25, 12.375, 12.0625, 10, 12.0625, 13.0625,
           12.1875, 11.25, 10.8125, 12.1875, 11.25, 12.4375, 11.9375, 12.4375,
           10.125, 12.75, 13.625, 12.4375, 11.875, 13.6875], dtype=bfloat16)
  ,
  {'l0': 66.0, 'loss': 2.015625},
  # jax.Array int32(30,) [≥4_838, ≤48_806] nonzero:30
    Array([43973, 14532, 14826, 35978, 17408, 23192, 47432, 48806, 10944,
           23018, 24860, 11840, 42230, 26714,  8659, 29402, 28714, 13640,
           19995, 22865, 39864,  4838, 29683, 14177, 17728, 24570, 38349,
           44021, 30253, 23287], dtype=int32)
  )}

In [ ]:
import pickle

with open("results_v8_2.pkl", "wb") as f:
    pickle.dump(results, f)

In [ ]:
import pickle
with open("results_v8_np.pkl", "rb") as f:
    results = pickle.load(f)


In [9]:
import numpy as np

results = {k: [np.array(x) for x in v] for k, v in results.items()}


array(51149, dtype=int32)

In [11]:
with open("results_v8_np.pkl", "wb") as f:
    pickle.dump(results, f)
# results

In [1]:
import pickle
with open("results_v8_np.pkl", "rb") as f:
    results = pickle.load(f)

In [5]:
results

{'en_es': (<jax.Array(6198, dtype=int32)>,
  <jax.Array(42.25, dtype=bfloat16)>,
  <jax.Array(47, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈4.7e+01 ±1.1 [≥4.2e+01, ≤4.8e+01] nonzero:30
    Array([47.5, 47.5, 47.5, 47.75, 47.5, 45, 47.5, 47, 47.5, 47.25, 45.25,
           47.5, 47, 47, 47.25, 45, 47.75, 47.5, 47.5, 42.25, 47.25, 47.25,
           47, 47.75, 47.25, 47.5, 47, 47.75, 47.25, 47.25], dtype=bfloat16)
  ,
  {'l0': 41.0, 'loss': 35.0},
  # jax.Array int32(30,) [≥2_060, ≤47_429] nonzero:30
    Array([ 3215, 25030, 24272, 21581, 47429, 44525, 29008, 27369, 28968,
           36766, 14166, 42753,  2060, 19895, 27192, 34292, 41973, 33789,
           14569,  6198,  3690, 41785,  8108, 11226, 34282, 31384,  6156,
           38707,  5088, 18168], dtype=int32)
  ),
 'en_it': (<jax.Array(6198, dtype=int32)>,
  <jax.Array(51.25, dtype=bfloat16)>,
  <jax.Array(52.75, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈5.3e+01 ±0.48 [≥5.1e+01, ≤5.4e+01] nonzero:30
    Array([53, 53, 53, 53.25, 51.5, 53.25, 52.75, 53, 53, 53, 53, 51.75,
           52.75, 52.5, 52.75, 52.75, 53.5, 52.75, 53, 52.75, 53.25, 52.75,
           51.25, 53, 52.75, 53, 52.75, 53, 53, 53], dtype=bfloat16)
  ,
  {'l0': 44.0, 'loss': 36.75},
  # jax.Array int32(30,) [≥666, ≤48_367] nonzero:30
    Array([ 1989, 34428,  3995, 29336, 44525, 30555, 23968,  8558, 19060,
           15949, 43877, 13468, 26671,  7896, 46283, 10937, 41973, 45067,
           17843, 29933, 19379, 20768,  6198, 36576, 48367, 25900,   666,
           47242, 17376, 12584], dtype=int32)
  ),
 'en_fr': (<jax.Array(5794, dtype=int32)>,
  <jax.Array(96, dtype=bfloat16)>,
  <jax.Array(97.5, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈9.7e+01 ±0.51 [≥9.6e+01, ≤9.8e+01] nonzero:30
    Array([97, 96, 97.5, 97, 97.5, 97.5, 96.5, 98, 98, 97.5, 97, 97, 97.5,
           96.5, 97.5, 97.5, 97.5, 97, 97.5, 97, 98, 97.5, 97, 97.5, 97.5, 97,
           97.5, 97, 98.5, 96.5], dtype=bfloat16)
  ,
  {'l0': 23.0, 'loss': 89.5},
  # jax.Array int32(30,) [≥12, ≤48_813] nonzero:30
    Array([29467,  5794,  4975,  6872, 26479, 48813, 28951, 23862, 44487,
            3700, 39357,  9522, 32883,  8023, 42720,    12, 20734, 28095,
            3242,   677, 22790, 25333, 34669, 27870, 20083,  7760,  6724,
            1780, 22333, 42771], dtype=int32)
  ),
 'en_de': (<jax.Array(5257, dtype=int32)>,
  <jax.Array(68, dtype=bfloat16)>,
  <jax.Array(69, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈6.9e+01 ±0.48 [≥6.8e+01, ≤7e+01] nonzero:30
    Array([68.5, 69, 69, 68.5, 68, 69, 69.5, 68.5, 68.5, 68.5, 68.5, 68.5,
           68.5, 69.5, 68.5, 69, 69, 68, 69, 68, 69, 69, 68, 69.5, 70, 69,
           68.5, 69, 69, 69], dtype=bfloat16)
  ,
  {'l0': 43.0, 'loss': 58.25},
  # jax.Array int32(30,) [≥1_609, ≤46_505] nonzero:30
    Array([19043, 12823, 14625,  6384,  5257, 46505, 21788, 21097, 24435,
           42263, 31586, 29976, 20768,  1609, 40653, 23254, 13687, 40751,
            9077, 16211, 32247, 20672, 33493,  9929, 41973, 30975, 42308,
            2537, 44499, 40721], dtype=int32)
  ),
 'en_ru': (<jax.Array(20738, dtype=int32)>,
  <jax.Array(85.5, dtype=bfloat16)>,
  <jax.Array(86.5, dtype=bfloat16)>,
  # jax.Array bfloat16(30,) ≈8.6e+01 ±0.5 [≥8.6e+01, ≤8.8e+01] nonzero:30
    Array([86.5, 86.5, 85.5, 86.5, 86, 86.5, 87, 86, 86.5, 86.5, 87, 87, 85.5,
           86.5, 86.5, 86.5, 86.5, 86, 86.5, 88, 87, 86, 86.5, 86, 86, 86.5,
           86, 87, 87, 86.5], dtype=bfloat16)
  ,
  {'l0': 70.0, 'loss': 73.0},
  # jax.Array int32(30,) [≥145, ≤47_388] nonzero:30
    Array([ 9677, 35326, 20738, 43181, 18692, 14593, 35421, 21169, 45708,
           19205,  4118,  5071, 26113, 40012, 40486, 32650,  2314, 39912,
            5829,  7192, 22031,  2077, 32101, 46427, 23266, 47388, 44884,
             145,  7272, 22790], dtype=int32)
  )}

In [58]:
with open("results_s_s.pkl", "wb") as f:
    import pickle
    pickle.dump(results, f)

In [ ]:
import pickle

with open("results.pkl", "wb") as f:
    pickle.dump(results, f)
# with open("results.json", "w") as f:
#     json.dump(results, f)

In [26]:
for task, (best_feature, best_loss, mean_loss, losses, metrics) in results.items():
    print(
        f"{task}, best feature {best_feature}, loss ratio {mean_loss / best_loss}, loss {best_loss}"
    )

algo_max, best feature 28655, loss ratio 1.61719, loss -0.816406
algo_min, best feature 42195, loss ratio 1.14062, loss -1.3125
algo_last, best feature 42756, loss ratio 1.46094, loss -0.839844
algo_first, best feature 39302, loss ratio 11.6875, loss -0.0683594
algo_sum, best feature 10061, loss ratio 1.22656, loss -1.08594
algo_most_common, best feature 3788, loss ratio 1.34375, loss -0.933594


In [ ]:
fs = FeatureSearch("en_de", 17)

--2024-05-22 00:38:44--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l17-test-run-6-4.52E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.125, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/1623d8da38be3171fcc8516a4cbe9fdb80e3d77e370aa5690895697649d688f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716597524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjU5NzUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvMTYyM2Q4ZGE

In [20]:
fs = FeatureSearch("en_ru", 18, early_stopping_steps=100, seed=10)
w = fs.find_weights()

--2024-05-22 01:20:39--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.51, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.51|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716600039&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjYwMDAzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

  0%|          | 0/2000 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f12cc5bf290>>
Traceback (most recent call last):
  File "/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [32]:
fs = FeatureSearch("algo_last", 18, early_stopping_steps=100, seed=10)

In [33]:
w = fs.find_weights()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [21]:
jax.lax.top_k(w[0], 20)

[# jax.Array float32(20,) ≈8.6 ±1.1e+01 [≥-0.055, ≤3.1e+01] nonzero:20
   Array([ 3.09979153e+01,  2.82771473e+01,  2.81605225e+01,  2.23171616e+01,
           1.70592213e+01,  1.41996422e+01,  1.22735662e+01,  1.18328285e+01,
           3.67472243e+00,  2.47404289e+00,  1.58222604e+00, -2.69895289e-02,
          -2.70592347e-02, -2.96332240e-02, -3.23766917e-02, -4.17804755e-02,
          -4.86377254e-02, -5.33655584e-02, -5.33791259e-02, -5.49723543e-02],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥6_262, ≤46_627] nonzero:20
   Array([10615, 41937, 10151, 13937, 16245, 46627,  6262, 16804,  8286,
          25117, 10897, 28142, 21109, 21014, 44885, 43632, 44120,  7628,
          25342, 36901], dtype=int32)
 ]

In [34]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(21195, dtype=int32)>,
 <jax.Array(-0.162109, dtype=bfloat16)>,
 <jax.Array(-0.980469, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-0.98 ±0.37 [≥-1.4, ≤-0.16] nonzero:20
   Array([-0.162109, -0.546875, -0.330078, -1.24219, -0.757812, -0.53125,
          -0.390625, -1.14844, -1.19531, -1.10156, -1.32812, -1.28125,
          -1.14844, -1.21875, -1.35938, -1.02344, -1.16406, -1.24219,
          -0.980469, -1.42188], dtype=bfloat16)
 )

In [ ]:
fs.check_features(jax.lax.top_k(w[0], 20)[1], scale=20)

  0%|          | 0/20 [00:00<?, ?it/s]

(<jax.Array(41239, dtype=int32)>,
 <jax.Array(-4.21875, dtype=bfloat16)>,
 <jax.Array(-5.34375, dtype=bfloat16)>,
 # jax.Array bfloat16(20,) ≈-5.3 ±0.37 [≥-6.0, ≤-4.2] nonzero:20
   Array([-6, -5.28125, -5.65625, -5.15625, -5.03125, -5.15625, -5.53125,
          -5.25, -5.28125, -5.65625, -5.1875, -5.71875, -5.59375, -5.0625,
          -4.21875, -5.40625, -5.0625, -5.53125, -5.21875, -5.65625],      dtype=bfloat16)
 )

In [ ]:
jax.lax.top_k(w, 20)

[# jax.Array float32(20,) ≈5.4 ±2.9 [≥-0.014, ≤1.1e+01] nonzero:20
   Array([11.022304  ,  9.960602  ,  9.723273  ,  7.446827  ,  7.3656707 ,
           6.9718046 ,  6.634519  ,  6.3051505 ,  6.268656  ,  6.1738625 ,
           5.3788037 ,  4.8796234 ,  4.4844356 ,  4.196526  ,  3.5582683 ,
           2.7230783 ,  1.9743043 ,  1.7151848 ,  1.7031406 , -0.01442439],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥3_151, ≤47_066] nonzero:20
   Array([ 9644, 20311, 46157, 42589, 25117, 38261,  9337, 47066, 10897,
          30124,  3151,  3398, 46627, 36916, 17215, 10069, 29942, 39270,
           4999, 31578], dtype=int32)
 ]

In [ ]:
def find_features(task, target_layer):
    scale = 20
    early_stopping_steps = 50
    init_w = 0.1
    iterations = 2000

    sae = get_sae(target_layer, 6)
    dictionary = sae["W_dec"]

    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    seed = 0

    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=seed+1)


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    target_tokens = [x[1] for x in tokenizer(dataset.completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)

    jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
    get_resids_initial = make_get_resids(llama, target_layer)
    get_resids_initial = jittify(get_resids_initial)

    intital_resids = get_resids_initial(inputs)

    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= target_layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    optimizer = optax.chain(
        optax.adam(3e-2),
        optax.zero_nans(),    
    )

    lwg = jax.value_and_grad(get_loss, has_aux=True)
    

    weights = jnp.ones(dictionary.shape[0]) * init_w
    opt_state = optimizer.init(weights)

    min_loss = 1e9
    early_stopping_counter = 0

    for _ in (bar := trange(iterations)):
        loss, weights, opt_state, metrics = train_step(weights, opt_state, dictionary, inputs, target_tokens, taker, intital_resids, optimizer, lwg)

        if metrics["loss"] < min_loss:
            min_loss = metrics["loss"]
            early_stopping_counter = 0
        

        tk = jax.lax.top_k(weights, 2)

        bar.set_postfix(loss_optim=loss, **metrics, top=tk[1][0], top_diff=(tk[0][0] - tk[0][1]) / tk[0][0])

        early_stopping_counter += 1
        if early_stopping_counter > early_stopping_steps:
            break

    
    return jax.lax.top_k(jnp.abs(weights), 20)


In [ ]:
find_features(
    "en_es", 18
)

  0%|          | 0/2000 [00:00<?, ?it/s]

[# jax.Array float32(20,) ≈3.3 ±4.8 [≥0.42, ≤1.6e+01] nonzero:20
   Array([16.070333  , 12.78331   , 10.591674  , 10.247323  ,  5.344763  ,
           2.8526528 ,  1.5353229 ,  0.5156362 ,  0.50892174,  0.45389727,
           0.45202777,  0.44730943,  0.44149902,  0.43828565,  0.4329766 ,
           0.43184933,  0.43073234,  0.4306618 ,  0.42252657,  0.4161794 ],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥2_719, ≤48_025] nonzero:20
   Array([10018, 20311,  5145, 40023, 45145,  2719, 15415, 24802, 20375,
          19505, 38618, 36786, 47611, 31726, 12632, 10615, 47480, 48025,
           3048, 40850], dtype=int32)
 ]

In [35]:
sae = get_sae(target_layer, 6)

--2024-05-22 18:45:26--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l17-test-run-6-4.52E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.125, 108.156.211.90, 108.156.211.95, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/1623d8da38be3171fcc8516a4cbe9fdb80e3d77e370aa5690895697649d688f3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716662726&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjY2MjcyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvMTYyM2Q4ZG

In [36]:
check_feature(21195, "algo_last", 18, sae)

NameError: name 'check_feature' is not defined

In [26]:
from micrlhf.utils.vector_storage import load_vector

task = "en_es"
layer = 18

tv = load_vector(f"task_vectors/{task}:{layer}.npz")

In [28]:
from micrlhf.utils.ito import grad_pursuit
from micrlhf.utils.load_sae import get_sae

sae = get_sae(18, 4)

w, r = grad_pursuit(tv, sae["W_dec"], 20, pos_only=True)

In [29]:
jax.lax.top_k(w, 20)

[# jax.Array float32(20,) ≈8.3 ±2.9 [≥5.0, ≤1.8e+01] nonzero:20
   Array([17.723856 , 11.587944 , 11.514933 , 10.496384 , 10.314079 ,
           9.238261 ,  9.234932 ,  8.850632 ,  7.3965535,  7.2235093,
           7.202634 ,  7.1291437,  6.861174 ,  6.529453 ,  6.5268745,
           6.065353 ,  6.048298 ,  5.570082 ,  5.558632 ,  4.9990907],      dtype=float32)
 ,
 # jax.Array int32(20,) [≥1_065, ≤48_437] nonzero:20
   Array([ 1065, 24363, 11831, 46849, 44525, 14577, 48437, 47882, 33953,
          34138, 24549, 28188, 39179, 28517, 43484, 35202, 33045, 48299,
           8787, 33039], dtype=int32)
 ]